# ValidMind for model validation — 112 Develop potential challenger models

Learn how to use ValidMind for your end-to-end model validation process based on common scenarios with our series of four introductory notebooks. In this second notebook, develop challenger models through a structured comparison process.

A *challenger model* is an alternate model that attempt to outperform the champion model, ensuring that the best performing fit-for-purpose model is always considered for deployment. Challenger models also help avoid over-reliance on a single model, and allow testing of new features, algorithms, or data sources without disrupting the production lifecycle.

## Prerequisites

In order to continue on the model validation journey with notebook, you'll need to first have:

- [ ] Registered a model within the ValidMind Platform and granted yourself access to the model as a validator
- [ ] Installed the ValidMind Library in your local environment, allowing you to access all its features

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;"><span style="color: #083E44;"><b>Need help with the above steps?</b></span>
<br></br>
Refer to the first notebook in this series: <a href="111-set_up_validmind.ipynb" style="color: #DE257E;"><b>111 Set up ValidMind</b></a></div>


## Setting up

### Initialize the ValidMind Library

First, let's connect up the ValidMind Library to our model we previously registered in the ValidMind Platform:

1. In a browser, [log in to ValidMind](https://docs.validmind.ai/guide/configuration/log-in-to-validmind.html).

2. In the left sidebar, navigate to **Inventory** and select the model you registered for this "ValidMind for model validation" series of notebooks.

3. Go to **Getting Started** and click **Copy snippet to clipboard**.

Next, [load your model identifier credentials from an `.env` file](https://docs.validmind.ai/developer/model-documentation/store-credentials-in-env-file.html) or replace the placeholder with your own code snippet:

In [ ]:
# Make sure the ValidMind Library is installed

%pip install -q validmind

# Load your model identifier credentials from an `.env` file

%load_ext dotenv
%dotenv .env

# Or replace with your code snippet

import validmind as vm

vm.init(
    # api_host="...",
    # api_key="...",
    # api_secret="...",
    # model="...",
)

### Import the champion model

Next, we'll import the champion model submitted by the model development team as we used in the last notebook (**[xgb_model_champion.pkl](xgb_model_champion.pkl)**) and load in the same sample [Lending Club](https://www.kaggle.com/datasets/devanshi23/loan-data-2007-2014/data) dataset:

In [ ]:
import xgboost as xgb

#Load the saved model
xgb_model = xgb.XGBClassifier()
xgb_model.load_model("xgb_model_champion.pkl")
xgb_model

# Ensure that we have to appropriate order in feature names from Champion model and dataset
cols_when_model_builds = xgb_model.get_booster().feature_names

In [ ]:
# Import the Lending Club dataset from Kaggle
from validmind.datasets.credit_risk import lending_club

df = lending_club.load_data(source="offline")
df.head()

# Preprocess the dataset for data quality testing purposes
preprocess_df = lending_club.preprocess(df)

# Apply feature engineering to the dataset
fe_df = lending_club.feature_engineering(preprocess_df)
fe_df.head()

## Split the feature engineered dataset

With our dummy model imported and our feature-engineered dataset ready to go, let's now **spilt our dataset into train and test** to start the validation testing process.

Splitting our dataset into train and test is essential for proper validation testing, as this helps assess how well the model generalizes to unseen data.

- We begin by dividing our data, which is based on Weight of Evidence (WoE) features, into training and testing sets (`train_df`, `test_df`).
- With `lending_club.split`, we employ a simple random split, randomly allocating data points to each set to ensure a mix of examples in both.

In [ ]:
# Split the data
train_df, test_df = lending_club.split(fe_df, test_size=0.2)

x_train = train_df.drop(lending_club.target_column, axis=1)
y_train = train_df[lending_club.target_column]

x_test = test_df.drop(lending_club.target_column, axis=1)
y_test = test_df[lending_club.target_column]

# Now let's apply the order of features from the champion model construction
x_train = x_train[cols_when_model_builds]
x_test = x_test[cols_when_model_builds]

In [ ]:
cols_use = ['annual_inc_woe',
 'verification_status_woe',
 'emp_length_woe',
 'installment_woe',
 'term_woe',
 'home_ownership_woe',
 'purpose_woe',
 'open_acc_woe',
 'total_acc_woe',
 'int_rate_woe',
 'sub_grade_woe',
 'grade_woe','loan_status']


train_df = train_df[cols_use]
test_df = test_df[cols_use]
test_df.head()

## Investigating potential challenger models*

Let's train two challenger models as basis for the testing

In [ ]:
# Define the Random Forest model
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(
    n_estimators=50, 
    random_state=42,
)

# Fit the model
rf_model.fit(x_train, y_train)

In [ ]:
#Second Challenger Model a Logistic Regression
from sklearn.linear_model import LogisticRegression

# Logistic Regression grid params
log_reg_params = {
    "penalty": ["l1", "l2"],
    "C": [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    "solver": ["liblinear"],
}

# Grid search for Logistic Regression
from sklearn.model_selection import GridSearchCV

grid_log_reg = GridSearchCV(LogisticRegression(), log_reg_params)
grid_log_reg.fit(x_train, y_train)

# Logistic Regression best estimator
log_reg = grid_log_reg.best_estimator_
log_reg

Compute probabilities as this is the raw probabilitistc output from the models of interest

In [ ]:
train_xgb_prob = xgb_model.predict_proba(x_train)[:, 1]
test_xgb_prob = xgb_model.predict_proba(x_test)[:, 1]

train_rf_prob = rf_model.predict_proba(x_train)[:, 1]
test_rf_prob = rf_model.predict_proba(x_test)[:, 1]

train_log_prob = log_reg.predict_proba(x_train)[:, 1]
test_log_prob = log_reg.predict_proba(x_test)[:, 1]

Compute binary predictions

In [ ]:
cut_off_threshold = 0.3 

train_xgb_binary_predictions = (train_xgb_prob > cut_off_threshold).astype(int)
test_xgb_binary_predictions = (test_xgb_prob > cut_off_threshold).astype(int)

train_rf_binary_predictions = (train_rf_prob > cut_off_threshold).astype(int)
test_rf_binary_predictions = (test_rf_prob > cut_off_threshold).astype(int)

train_log_binary_predictions = (train_log_prob > cut_off_threshold).astype(int)
test_log_binary_predictions = (test_log_prob > cut_off_threshold).astype(int)

## Initializing the ValidMind objects*

### Initialize the ValidMind datasets

Before you can run tests, you must first initialize a ValidMind dataset object using the [`init_dataset`](https://docs.validmind.ai/validmind/validmind.html#init_dataset) function from the ValidMind (`vm`) module.

This function takes a number of arguments:

- `dataset`: The dataset that you want to provide as input to tests.
- `input_id`: A unique identifier that allows tracking what inputs are used when running each individual test.
- `target_column`: A required argument if tests require access to true values. This is the name of the target column in the dataset.

With all datasets ready, you can now initialize the raw, processed, training and test datasets (`raw_df`, `preprocessed_df`, `fe_df`,  `train_df` and `test_df`) created earlier into their own dataset objects using [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset):

In [ ]:
vm_raw_dataset = vm.init_dataset(
    dataset=df,
    input_id="raw_dataset",
    target_column=lending_club.target_column,
)

vm_preprocess_dataset = vm.init_dataset(
    dataset=preprocess_df,
    input_id="preprocess_dataset",
    target_column=lending_club.target_column,
)

vm_fe_dataset = vm.init_dataset(
    dataset=fe_df,
    input_id="fe_dataset",
    target_column=lending_club.target_column,
)

vm_train_ds = vm.init_dataset(
    dataset=train_df,
    input_id="train_dataset",
    target_column=lending_club.target_column,
)

vm_test_ds = vm.init_dataset(
    dataset=test_df,
    input_id="test_dataset",
    target_column=lending_club.target_column,
)

### Initialize a model object

You will also need to initialize a ValidMind model object (`vm_model`) that can be passed to other functions for analysis and tests on the data. You simply intialize this model object with [`vm.init_model()`](https://docs.validmind.ai/validmind/validmind.html#init_model):

In [ ]:
vm_xgb_model = vm.init_model(
    xgb_model,
    input_id="xgb_model_developer_champion",
)

vm_rf_model = vm.init_model(
    rf_model,
    input_id="rf_model",
)

vm_log_model = vm.init_model(
    log_reg,
    input_id="log_model",
)


### Assign prediction values and probabilities to the datasets

With our model now trained, we'll move on to assigning both the predictive probabilities coming directly from the model's predictions, and the binary prediction after applying the cutoff threshold described in the previous steps. 
- These tasks are achieved through the use of the `assign_predictions()` method associated with the VM `dataset` object.
- This method links the model's class prediction values and probabilities to our VM train and test datasets.

In [ ]:
# XGBoost Model
vm_train_ds.assign_predictions(
    model=vm_xgb_model,
    prediction_values=train_xgb_binary_predictions,
    prediction_probabilities=train_xgb_prob,
)

vm_test_ds.assign_predictions(
    model=vm_xgb_model,
    prediction_values=test_xgb_binary_predictions,
    prediction_probabilities=test_xgb_prob,
)

# Random Forest Model
vm_train_ds.assign_predictions(
    model=vm_rf_model,
    prediction_values=train_rf_binary_predictions,
    prediction_probabilities=train_rf_prob,
)

vm_test_ds.assign_predictions(
    model=vm_rf_model,
    prediction_values=test_rf_binary_predictions,
    prediction_probabilities=test_rf_prob,
)


# Logistic Regression 
vm_train_ds.assign_predictions(
    model=vm_log_model,
    prediction_values=train_log_binary_predictions,
    prediction_probabilities=train_log_prob,
)

vm_test_ds.assign_predictions(
    model=vm_log_model,
    prediction_values=test_log_binary_predictions,
    prediction_probabilities=test_log_prob,
)

### Compute credit risk scores

In this phase, we translate model predictions into actionable scores using probability estimates generated by our trained model.

In [ ]:
train_xgb_scores = lending_club.compute_scores(train_xgb_prob)
test_xgb_scores = lending_club.compute_scores(test_xgb_prob)
train_rf_scores = lending_club.compute_scores(train_rf_prob)
test_rf_scores = lending_club.compute_scores(test_rf_prob)
train_log_scores = lending_club.compute_scores(train_log_prob)
test_log_scores = lending_club.compute_scores(test_log_prob)

# Assign scores to the datasets
vm_train_ds.add_extra_column("xgb_scores", train_xgb_scores)
vm_test_ds.add_extra_column("xgb_scores", test_xgb_scores)
vm_train_ds.add_extra_column("rf_scores", train_rf_scores)
vm_test_ds.add_extra_column("rf_scores", test_rf_scores)
vm_train_ds.add_extra_column("log_scores", train_log_scores)
vm_test_ds.add_extra_column("log_scores", test_log_scores)

# WIP BELOW**

### Enable custom context for test descriptions

When you run ValidMind tests, test descriptions are automatically generated with LLM using the test results, the test name, and the static test definitions provided in the test’s docstring. While this metadata offers valuable high-level overviews of tests, insights produced by the LLM-based descriptions may not always align with your specific use cases or incorporate organizational policy requirements.

Here, we'll include some initial custom use case context, improving the relevancy, insight, and format of the test descriptions returned.

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;"><span style="color: #083E44;"><b>Want to learn more about setting custom context for LLM-generated test descriptions?</b></span>
<br></br>
Refer to our extended walkthrough notebook: <a href="https://docs.validmind.ai/notebooks/how_to/add_context_to_llm_descriptions.html" style="color: #DE257E;"><b>Add context to LLM-generated test descriptions
</b></a></div>

#### Review default LLM-generated descriptions

By default, custom context for LLM-generated descriptions is disabled, meaning that the output will not include any additional context.

Let's generate an initial test description for the `HighPearsonCorrelation` test for comparison with later iterations:

In [ ]:
vm.tests.run_test(
    "validmind.data_validation.HighPearsonCorrelation",
    inputs={
        "dataset": fe_df,
    },
)

#### Enable use case context

To enable custom use case context, set the `VALIDMIND_LLM_DESCRIPTIONS_CONTEXT_ENABLED` environment variable to `1`.

This is a global setting that will affect all tests for your linked model:

In [ ]:
import os
os.environ["VALIDMIND_LLM_DESCRIPTIONS_CONTEXT_ENABLED"] = "1"

Enabling use case context allows you to pass in additional context to the LLM-generated text descriptions within `context`:

In [ ]:
context = """
FORMAT FOR THE LLM DESCRIPTIONS: 
    **<Test Name>** is designed to <begin with a concise overview of what the test does and its primary purpose, 
    extracted from the test description>.

    The test operates by <write a paragraph about the test mechanism, explaining how it works and what it measures. 
    Include any relevant formulas or methodologies mentioned in the test description.>

    The primary advantages of this test include <write a paragraph about the test's strengths and capabilities, 
    highlighting what makes it particularly useful for specific scenarios.>

    Users should be aware that <write a paragraph about the test's limitations and potential risks. 
    Include both technical limitations and interpretation challenges. 
    If the test description includes specific signs of high risk, incorporate these here.>

    **Key Insights:**

    The test results reveal:

    - **<insight title>**: <comprehensive description of one aspect of the results>
    - **<insight title>**: <comprehensive description of another aspect>
    ...

    Based on these results, <conclude with a brief paragraph that ties together the test results with the test's 
    purpose and provides any final recommendations or considerations.>

ADDITIONAL INSTRUCTIONS:
    Present insights in order from general to specific, with each insight as a single bullet point with bold title.
    You are a model validator and the goal is to identify risk and/or suggest room for improvements or recommendations on what Model Developer should do in order to improve outcomes and reduce risk

    For each metric in the test results, include in the test overview:
    - The metric's purpose and what it measures
    - Its mathematical formula
    - The range of possible values
    - What constitutes good/bad performance
    - How to interpret different values

    Each insight should progressively cover:
    1. Overall scope and distribution
    2. Complete breakdown of all elements with specific values
    3. Natural groupings and patterns
    4. Comparative analysis between datasets/categories
    5. Stability and variations
    6. Notable relationships or dependencies

    Remember:
    - Keep all insights at the same level (no sub-bullets or nested structures)
    - Make each insight complete and self-contained
    - Include specific numerical values and ranges
    - Cover all elements in the results comprehensively
    - Maintain clear, concise language
    - Use only "- **Title**: Description" format for insights
    - Progress naturally from general to specific observations

""".strip()

os.environ["VALIDMIND_LLM_DESCRIPTIONS_CONTEXT"] = context

With the use case context set, generate an updated test description for the `DatasetDescription` test for comparision with default output:

In [ ]:
vm.tests.run_test(
    "validmind.data_validation.HighPearsonCorrelation",
    inputs={
        "dataset": fe_df,
    },
)

## In summary

## Next steps